<a href="https://colab.research.google.com/github/ViruniRuvinya/Image-Classification---CNN/blob/main/Cat_dog_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opendatasets -q

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import time
import opendatasets as od

In [5]:
od.download('https://www.kaggle.com/datasets/dineshpiyasamara/cats-and-dogs-for-classification')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: viruniruvinya
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/dineshpiyasamara/cats-and-dogs-for-classification


100%|██████████| 217M/217M [00:12<00:00, 18.6MB/s]


In [8]:
Batch_size=32
Image_size=(128,128)

In [7]:
train_data_dir='/content/cats-and-dogs-for-classification/cats_dogs/train'
test_data_dir='/content/cats-and-dogs-for-classification/cats_dogs/test'

In [10]:
train_data=tf.keras.utils.image_dataset_from_directory(train_data_dir,
                                                       batch_size=Batch_size,
                                                       image_size=Image_size,
                                                       subset='training',
                                                       validation_split=0.1,
                                                       seed=42)

validation_data=tf.keras.utils.image_dataset_from_directory(train_data_dir,
                                                       batch_size=Batch_size,
                                                       image_size=Image_size,
                                                       subset='validation',
                                                       validation_split=0.1,
                                                       seed=42)

test_data=tf.keras.utils.image_dataset_from_directory(test_data_dir,
                                                       batch_size=Batch_size,
                                                       image_size=Image_size)

Found 8000 files belonging to 2 classes.
Using 7200 files for training.
Found 8000 files belonging to 2 classes.
Using 800 files for validation.
Found 2000 files belonging to 2 classes.


In [11]:
class_names=train_data.class_names
class_names

['cats', 'dogs']

In [13]:
for image_batch,label_batch in train_data.take(1):
  print(image_batch.shape)
  print(label_batch.shape)

(32, 128, 128, 3)
(32,)
